In [1]:
from model import *
import pytchat


C:\Users\avika\OneDrive\Documents\UAL\interactive_dance_thesis


In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="michellejieli/emotion_text_classifier")
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")
model = AutoModelForSequenceClassification.from_pretrained("michellejieli/emotion_text_classifier")

In [3]:


chat = pytchat.create(video_id="jfKfPfyJRdk")
while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
        result = pipe(c.message)
        print(result)

2023-10-18 20:00:52 [Рамил Мирзоев]- Эля
[{'label': 'neutral', 'score': 0.9787927269935608}]
2023-10-18 20:00:53 [Junaid]- H- yeah we’re everywhere i guess
[{'label': 'neutral', 'score': 0.9099355936050415}]
2023-10-18 20:00:54 [beyza]- Ted. how are you
[{'label': 'neutral', 'score': 0.95458984375}]
2023-10-18 20:00:56 [Petra Miller]- INC U
[{'label': 'neutral', 'score': 0.9838861227035522}]


In [2]:
# Preparing MEED data for model
MEED= prep_MEED_data()
train_data,val_data,frame_dim,max_x,min_x,max_y,min_y,max_dx,min_dx,max_dy,min_dy = MEED

# create model
m = MotionModel(input_dim=frame_dim, output_dim=frame_dim, blocksize=BLOCK_SIZE, hidden_dim=512, n_layers=8, dropout=DROPOUT)
m = m.to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=LEARNING_RATE, weight_decay=L2_REG)  # weight_decay=1e-5 L2 regularization
m, optimizer, epoch, loss, train_seed = load_checkpoint(m, optimizer, CHECKPOINT_PATH)
# Generate a sequence
print(f'Generating sequence of {FRAMES_GENERATE} frames...')
BATCH_SIZE = 1
xb,yb,mask = get_batch('val',BLOCK_SIZE,BATCH_SIZE, train_data,val_data)
generated = m.generate(xb, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)


100%|██████████| 4102/4102 [00:03<00:00, 1086.69it/s]


Validating interpolation...
No errors found!


4102it [00:01, 2499.12it/s]
4102it [00:01, 2397.84it/s]


Creating keypoint frames...


100%|██████████| 4102/4102 [00:03<00:00, 1039.36it/s]


...

4102it [00:03, 1110.06it/s]


Validating length of data after delta
Adding emotions to frames...


100%|██████████| 4102/4102 [00:00<00:00, 9863.41it/s]


Validating length of data after emotions
frame_dim: 107
Loading checkpoint...
Checkpoint loaded from checkpoints/proto5_checkpoint.pth
Generating sequence of 300 frames...


NameError: name 'batch' is not defined

In [19]:
def encoding_to_emotions(encoding):
    """
    Convert an encoding to its corresponding emotion labels.
    
    Parameters:
    - encoding (list of int): The encoding of the emotions.
    
    Returns:
    - list of str: The labels of the emotions.
    """
    
    emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprise']
    
    # Find the indices of non-zero values in the encoding and return the corresponding labels
    emotion_indices = [i for i, value in enumerate(encoding) if value > 0]
    return [emotion_labels[i] for i in emotion_indices]

# Test the function with an example encoding
test_encoding = [0.97, 0, 1, 1, 0, 0, 0]
encoding_to_emotions(test_encoding)


['Anger', 'Fear', 'Happiness']

In [12]:
frame[-7:]

[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]

In [33]:
# switch emotions of frames 
def switch_emotion(frame,old_emotion,new_emotion):
    """
    Switch the emotion of a frame.
    
    Parameters:
    - frame (list of int): The frame to switch the emotion of.
    - old_emotion (str): The emotion to switch from.
    - new_emotion (str): The emotion to switch to.
    
    Returns:
    - list of int: The frame with the emotion switched.
    """
    
    # Get the indices of the old and new emotion
    emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprise']
    old_emotion_index = emotion_labels.index(old_emotion)
    new_emotion_index = emotion_labels.index(new_emotion)
    
    # Switch the emotion
    frame[-7 + old_emotion_index] = 0
    frame[-7 + new_emotion_index] = 1
    
    return frame

new_batch = []
for frame in batch:
    emotion = encoding_to_emotions(frame[-7:])
    print(emotion)
    new_batch.append(frame[:-7] + emotion_to_encoding('Disgust'))

['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Happiness']
['Anger', 'Disgust', 'Happiness', 'Neutral']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Happiness', 'Neutral', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Sad', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Fear', 'Happiness']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Surprise']
['Anger', 'Disgust', 'Happiness', 'Sur

In [24]:
for batch in unnorm_out:
    visualise_skeleton(batch, max_x, max_y, max_frames=FRAMES_GENERATE,save =True,save_path=None,prefix=f'adam_{EPOCHS}_coord_normalemotion_happy',train_seed=train_seed,delta=False)

Visualizing frames: 100%|██████████| 300/300 [00:33<00:00,  9.07it/s]


In [25]:
visualise_skeleton(new_batch, max_x, max_y, max_frames=FRAMES_GENERATE,save = True,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytosad',train_seed=train_seed,delta=False)

Visualizing frames:  27%|██▋       | 82/300 [00:09<00:24,  9.06it/s]


KeyboardInterrupt: 

In [34]:
happy_to_sad_xb = [new_batch[-16:]]
happy_to_sad_xb = torch.tensor(happy_to_sad_xb).to(device).float()


generated = m.generate(happy_to_sad_xb, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)


In [35]:
for new_batch in unnorm_out:

    visualise_skeleton(new_batch, max_x, max_y, max_frames=FRAMES_GENERATE,save = True,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust',train_seed=train_seed,delta=False)

Visualizing frames: 100%|██████████| 300/300 [00:32<00:00,  9.17it/s]
